# Knock airport Weather

### assignment_6_Weather.ipynb

![knock](img/knockweather_%2020241214_210223.jpg)

***

Get the data from this link.

https://cli.fusio.net/cli/climate_data/webdata/hly4935.csv


Plot:

The temperature
The mean temperature each day
The mean temperature for each month

## Libraries

***

We use pandas for the dataFrame data structure.

It allows to investigate [CSV](https://pandas.pydata.org/docs/user_guide/10min.html) files, amongst other features.

[matplotlib](https://www.w3schools.com/python/matplotlib_pyplot.asp) is theplotting library in python that serves as a 
visualization utility.

[NumPy](https://www.w3schools.com/python/numpy/default.asp) to help us working with arrays.

In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


### Load data and clean dataset

***

#### Load the weather data from URL.

In [37]:
# Load the dataset
df = pd.read_csv('./hly4935.csv', skiprows=23, low_memory=False)

# Remove invalid date entries (optional: inspect for common issues)
df['date'] = df['date'].str.strip()  # Remove extra spaces
df['date'] = pd.to_datetime(df['date'], format='%d-%b-%Y %H:%M:%S', errors='coerce')

# Drop rows with unparseable dates
df = df.dropna(subset=['date'])


# Rename and clean relevant columns based on your mapping
column_mapping = {
    'date': 'date',
    'temp': 'temperature',
    # Include other columns you want to clean or use
}
df.rename(columns=column_mapping, inplace=True)

# Convert date column to datetime format
df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')

# Drop rows with invalid dates or missing temperatures
df = df.dropna(subset=['date', 'temperature'])

# Set the date as the index for easy resampling
df.set_index('date', inplace=True)


#### Ensure Numerical Columns are Cleaned
***

In [41]:
# Clean numerical columns
numerical_columns = ['temperature']  # Add other relevant columns as needed
for col in numerical_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with missing temperature
df = df.dropna(subset=numerical_columns)


## Plot:

The temperature

The mean temperature each day

The mean temperature for each month

***



## Plot:

The Windspeed (there is data missing from this column)

The rolling windspeed (say over 24 hours)

The max windspeed for each day

The monthly mean of the daily max windspeeds


***

## End

***